<a href="https://colab.research.google.com/github/OumaymaRedissi/Simulation/blob/main/Chargers_lxml_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install lxml
from lxml import etree
import pandas as pd 
#help(etree)

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
chargers= etree.Element("chargers")
charger = etree.SubElement(chargers, "charger", id="401301237" ,x="4468413.2" ,y="5335744.5",plug_power="22" ,plug_count="2")
etree.tostring(charger)

b'<charger id="401301237" plug_count="2" plug_power="22" x="4468413.2" y="5335744.5"/>'

In [3]:
import xml.etree.ElementTree as ET
# parsing directly.
tree = ET.parse('charging_stations.xml')
root = tree.getroot()
# printing the root.
print(root)

<Element 'osm' at 0x7fca0296def0>


In [4]:
def is_float(element) -> bool:
    try:
        float(element)
        return True
    except ValueError:
        return False

**pp_formatter()** to remove the white spaces and deletes everything after the 'kw'

In [5]:
def pp_formatter(plug_power:str) -> str:
    start_index = plug_power.find("kW")
    plug_power = (plug_power[0: start_index:] + plug_power[len(plug_power) + 1::]).replace(' ','')
    return plug_power
    

In [6]:
def extractor() :
  df = pd.DataFrame([],['id','x','y','plug_power','plug_count'])
  for node in root.findall('node'):
      id = node.get('id')
      lat = node.get('lat')
      lon = node.get('lon')
      capacity=2 #default capacity =2
      list_of_plug_powers=[]
      for tag in node.findall('tag'):
        att_k=tag.get('k')
        att_v=tag.get('v')
        if att_k == "capacity":
          if is_float(att_v):
            capacity = int(att_v)
        if "kW" in att_v :
          plug_power_str = pp_formatter(att_v)
          if is_float(plug_power_str):
            list_of_plug_powers.append(float(plug_power_str))
      plug_power = max(list_of_plug_powers, default=22)#default plug power=22
      #print(id,lat,lon,plug_power,capacity)
      data = {'id':id,'x':lat,'y':lon,'plug_power':plug_power,'plug_count':capacity}
      df = df.append(data,ignore_index=True)
  
  #print(df)
  return df
       

print(extractor())

              id           x          y  plug_power  plug_count
0            NaN         NaN        NaN         NaN         NaN
1            NaN         NaN        NaN         NaN         NaN
2            NaN         NaN        NaN         NaN         NaN
3            NaN         NaN        NaN         NaN         NaN
4            NaN         NaN        NaN         NaN         NaN
...          ...         ...        ...         ...         ...
1455  9791175803  48.8555664  2.3150384        22.0         6.0
1456  9792084981  49.1797976  3.1234451        22.0         2.0
1457  9797611182  48.8994232  2.3710185        22.0         6.0
1458  9807217413  48.8541853  2.3452424        22.0         5.0
1459  9811956662  48.7095969  2.1710766        22.0         2.0

[1460 rows x 5 columns]


In [7]:
df = extractor()
df.to_csv('file_name.csv')